# CHAPTER 9. 텐서플로 시작하기

...

...

...

---
## 9.1 설치

---
## 9.2 첫 번째 계산 그래프를...

In [10]:
np.random.seed(4) # 난수 생성을 위한 시드 설정
m = 60
w1, w2 = 0.1, 0.3
noise = 0.1

angles = np.random.rand(m) * 3 * np.pi / 2 - 0.5
X = np.empty((m, 3))
X[:, 0] = np.cos(angles) + np.sin(angles)/2 + noise * np.random.randn(m) / 2
X[:, 1] = np.sin(angles) * 0.7 + noise * np.random.randn(m) / 2
X[:, 2] = X[:, 0] * w1 + X[:, 1] * w2 + noise * np.random.randn(m)